In [95]:
import pandas as pd
import numpy as np
# 用於二元目標變數
from scipy.stats import pointbiserialr
from functools import reduce
from sklearn.model_selection import TimeSeriesSplit, cross_val_score, GridSearchCV

In [2]:
# Import GDP Data
from data import OverallGeneralEcon, df, MacroInfo, JuridicalInfo

c:\Users\user1\Desktop\repo2\revenue-prediction\data.py:48: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  Dream_report = pd.read_sql(qr_sub, conn)
c:\Users\user1\Desktop\repo2\revenue-prediction\data.py:72: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  stock_info = pd.read_sql(stock_qr_sub, conn)
c:\Users\user1\Desktop\repo2\revenue-prediction\data.py:95: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  MacroInfo = pd.read_sql(macro_qr_sub, conn)


            年月     代號           數值                  名稱
0       202412  19600        1.060       經濟成長率(GDP)–單季
1       202412  19700   201644.000      國內生產毛額(GDP)–美元
2       202412  19710  6404219.000      國內生產毛額(GDP)–台幣
3       202412  19750     8567.000  平均每人國內生產毛額(GDP)–美元
4       202412  19760   272086.000  平均每人國內生產毛額(GDP)–台幣
...        ...    ...          ...                 ...
263426  201401  USA96      954.642              美國營建支出
263427  201401  USA97        0.000           美國新屋開工年增率
263428  201401  USA98     1723.614             美國企業存貨額
263429  201401  USA99        1.600          美國核心CPI年增率
263430  201401  ZAF00      100.300        OECD領先指標(南非)

[263431 rows x 4 columns]


c:\Users\user1\Desktop\repo2\revenue-prediction\data.py:107: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  JuridicalInfo = pd.read_sql(juridical_qr_sub, conn)


In [6]:
# 名目、實質、時間
"""
MacroInfo
名目有效匯率指數–台灣
台灣消費者信心指數
台灣消費者信心指數–未來半年國內經濟景氣
台灣消費者信心指數–未來半年國內物價水準
美國消費者信心指數
經濟成長率(GDP)–全年
--
實質國內生產毛額(GDP)–台幣 -> 只更新到202009
實質GDP成長率(台灣) -> 只更新到202212
--
貨幣總額(M1A)日平均、貨幣總額(M2)日平均、貨幣總額年增率(M1A)日平均
臺灣製造業PMI - 新增出口訂單 - 基礎原物料
台灣外銷訂單金額–全球
消費者物價指數(CPI)、消費者物價指數(CPI)年增率、核心消費者物價指數
美國核心PPI、美國核心PPI月增率
進口物價指數、出口物價指數年增率

sysallunTWESPF
外銷訂單_美金計價(百萬美元)_合計_年增率(%)(月)
景氣同時指標_工業生產指數(Index 2006=100)_年增率(%)(月)

sysTaiStaValMAC
景氣指標統計_景氣領先指標不含趨勢指數(點)_年增率(%)(月)

其他
財報、營收
"""

'\n名目有效匯率指數–台灣\n台灣消費者信心指數\n台灣消費者信心指數–未來半年國內經濟景氣\n台灣消費者信心指數–未來半年國內物價水準\n美國消費者信心指數\n經濟成長率(GDP)–全年\n實質國內生產毛額(GDP)–台幣\n實質GDP成長率(台灣)\n貨幣總額(M1A)日平均、貨幣總額(M2)日平均、貨幣總額年增率(M1A)日平均\n臺灣製造業PMI - 新增出口訂單 - 基礎原物料\n台灣外銷訂單金額–全球\n消費者物價指數(CPI)、消費者物價指數(CPI)年增率、核心消費者物價指數\n美國核心PPI、美國核心PPI月增率\n進口物價指數、出口物價指數年增率\n'

### 總經資料處理

In [118]:
# Improt data
# 名目有效匯率指數
NominalExchageRate = MacroInfo[MacroInfo['名稱']=='名目有效匯率指數–台灣'][['年月','數值','名稱']].rename(columns={'年月':'ddate', '數值':'value', '名稱':'category'})
display(NominalExchageRate.head(3))

# 台灣消費者信心指數
TW_ICS = MacroInfo[MacroInfo['名稱']=='台灣消費者信心指數'][['年月','數值','名稱']].rename(columns={'年月':'ddate', '數值':'value', '名稱':'category'})
display(TW_ICS.tail(3))

# 消費者物價指數(CPI)年增率
TW_CPIGrowth = MacroInfo[MacroInfo['名稱']=='消費者物價指數(CPI)年增率'][['年月','數值','名稱']].rename(columns={'年月':'ddate', '數值':'value', '名稱':'category'})
display(TW_CPIGrowth.head(3))

# 經濟成長率(GDP)–全年
TW_GDPGrowth = MacroInfo[MacroInfo['名稱']=='經濟成長率(GDP)–單季'][['年月','數值','名稱']].rename(columns={'年月':'ddate', '數值':'value', '名稱':'category'})
display(TW_GDPGrowth.head(3))

,ddate,value,category
1850,202311,101.05,名目有效匯率指數–台灣
3066,202310,100.99,名目有效匯率指數–台灣
4867,202309,101.07,名目有效匯率指數–台灣


,ddate,value,category
258631,201403,80.96,台灣消費者信心指數
260946,201402,82.93,台灣消費者信心指數
263241,201401,80.83,台灣消費者信心指數


,ddate,value,category
309,202312,2.71,消費者物價指數(CPI)年增率
1170,202311,2.90,消費者物價指數(CPI)年增率
2326,202310,3.05,消費者物價指數(CPI)年增率


,ddate,value,category
0,202412,1.06,經濟成長率(GDP)–單季
27,202409,2.12,經濟成長率(GDP)–單季
42,202406,4.77,經濟成長率(GDP)–單季


In [119]:
#JuridicalInfo.columns
#上市買賣超金額(百萬)、上櫃買賣超金額(百萬)、合計買進金額(百萬)

In [120]:
TW_NominalGDP = OverallGeneralEcon[['time','Nominal_GDP']]
TW_NominalGDP = TW_NominalGDP.rename(columns={'time':'ddate'})
#print(TW_NominalGDP.head(3))
TW_NominalGDP['ddate'] = TW_NominalGDP['ddate'].astype(str)

In [121]:
# 將GDP季資料先合併
GDP_quarterly = pd.merge(TW_NominalGDP, TW_GDPGrowth, on='ddate', how='left')
GDP_quarterly.columns = ['ddate', 'NominalGDP', 'GDP_growth', 'category_GDPgrowth']
#GDP_quarterly.head(3)

In [122]:
# (月)總經資料
dfs = [NominalExchageRate, TW_ICS, TW_CPIGrowth]
MergedMacroData = reduce(lambda left, right: pd.merge(left, right, on='ddate', how='left'), dfs)

# Rename columns
MergedMacroData.columns = ['ddate', 'FX', 'category_FX', 'CCI',
                          'category_CCI', 'CPI_growth', 'category_CPIgrowth']

#display(MergedMacroData)
# FX-Foriegn-exchange rate
# CCI-台灣消費者信心指數

In [123]:
MergedMacroData = pd.merge(MergedMacroData, GDP_quarterly, on='ddate', how='left')

In [125]:
MergedMacroData['NominalGDP'] = MergedMacroData['NominalGDP'].ffill()
MergedMacroData['GDP_growth'] = MergedMacroData['GDP_growth'].ffill()
MergedMacroData['category_GDPgrowth'] = MergedMacroData['category_GDPgrowth'].ffill()

In [126]:
MergedMacroData.head(9)

,ddate,FX,category_FX,CCI,category_CCI,CPI_growth,category_CPIgrowth,NominalGDP,GDP_growth,category_GDPgrowth
0,202311,101.05,名目有效匯率指數–台灣,69.40,台灣消費者信心指數,2.90,消費者物價指數(CPI)年增率,NaN,NaN,NaN
1,202310,100.99,名目有效匯率指數–台灣,69.06,台灣消費者信心指數,3.05,消費者物價指數(CPI)年增率,NaN,NaN,NaN
2,202309,101.07,名目有效匯率指數–台灣,66.73,台灣消費者信心指數,2.93,消費者物價指數(CPI)年增率,6093017.0,2.54,經濟成長率(GDP)–單季
3,202308,100.67,名目有效匯率指數–台灣,67.51,台灣消費者信心指數,2.53,消費者物價指數(CPI)年增率,6093017.0,2.54,經濟成長率(GDP)–單季
4,202307,101.31,名目有效匯率指數–台灣,68.39,台灣消費者信心指數,1.88,消費者物價指數(CPI)年增率,6093017.0,2.54,經濟成長率(GDP)–單季
5,202306,102.80,名目有效匯率指數–台灣,66.66,台灣消費者信心指數,1.75,消費者物價指數(CPI)年增率,5697026.0,1.36,經濟成長率(GDP)–單季
6,202305,101.84,名目有效匯率指數–台灣,63.67,台灣消費者信心指數,2.02,消費者物價指數(CPI)年增率,5697026.0,1.36,經濟成長率(GDP)–單季
7,202304,101.35,名目有效匯率指數–台灣,63.18,台灣消費者信心指數,2.35,消費者物價指數(CPI)年增率,5697026.0,1.36,經濟成長率(GDP)–單季
8,202303,101.85,名目有效匯率指數–台灣,64.47,台灣消費者信心指數,2.35,消費者物價指數(CPI)年增率,5482976.0,-3.31,經濟成長率(GDP)–單季


### 單純以總經變數訓練模型

In [128]:
# Split data by TimeSeriesSplit
tscv_macro = TimeSeriesSplit(n_splits=50)

# each split contains train set & test set 'index'
splits_macro = list(tscv_macro.split(MergedMacroData))

In [104]:
# 使用最後一組的 trained model index 切分 training & testing data
train_idx_macro = splits_macro[-1][0]
test_idx_macro = splits_macro[-1][1]

In [107]:
df = df.rename(columns={'direction':'real_direction'})

,ddate,stockid,real_direction,rev,mom,yoy,revd1,yoyd1,mond1,revd2,...,mond9,revd10,yoyd10,mond10,revd11,yoyd11,mond11,revd12,yoyd12,mond12
0,202310,1101,1,9371746.0,7.29,-17.56,8735157.0,-16.05,-6.87,9379408.0,...,-41.79,12584154.0,16.29,30.07,9674576.0,-0.80,-14.90,11368096.0,18.85,9.26
1,202310,1102,1,6706168.0,0.21,-11.77,6691905.0,-4.32,3.43,6469836.0,...,-42.28,8340507.0,-11.31,0.41,8306062.0,-7.56,9.27,7601097.0,-19.60,8.68
2,202310,1103,1,257343.0,10.46,22.88,232979.0,13.50,-2.49,238917.0,...,-16.37,228644.0,33.31,3.10,221763.0,12.13,5.89,209429.0,8.43,2.03
3,202310,1104,1,654514.0,5.81,3.08,618592.0,3.59,0.23,617197.0,...,-30.44,725055.0,27.29,14.76,631827.0,6.68,-0.50,634981.0,13.60,6.33
4,202310,1108,1,451146.0,11.62,12.45,404171.0,16.60,-6.10,430420.0,...,-29.50,418326.0,31.04,9.00,383773.0,43.18,-4.34,401202.0,44.98,15.74
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205195,201401,9951,1,313481.0,45.11,55.35,216032.0,22.01,-14.53,252765.0,...,1.32,240746.0,25.89,43.33,167967.0,1.81,-16.76,201785.0,43.73,13.96
205196,201401,9955,0,271410.0,-26.84,-31.20,370962.0,23.48,121.37,167575.0,...,31.64,428478.0,-39.45,11.82,383183.0,-41.55,-2.87,394489.0,-37.48,31.31
205197,201401,9958,0,165307.0,-47.12,-16.91,312585.0,-17.06,14.16,273803.0,...,-20.03,351222.0,56.20,111.12,166364.0,-31.57,-16.38,198944.0,49.34,-47.21
205198,201401,9960,0,40678.0,-26.86,-22.57,55616.0,20.42,-4.63,58313.0,...,15.18,61642.0,-49.52,46.99,41935.0,-14.70,-20.18,52534.0,6.78,13.74


In [137]:
# 合併df以及總經資料
TheWhole = pd.merge(df[['ddate', 'stockid', 'real_direction']].astype({'ddate':'str'}), 
                    MergedMacroData, on='ddate', how='left')
#TheWhole[TheWhole['ddate']=='202302'].head(3)

,ddate,stockid,real_direction,FX,category_FX,CCI,category_CCI,CPI_growth,category_CPIgrowth,NominalGDP,GDP_growth,category_GDPgrowth
14410,202302,1101,0,102.15,名目有效匯率指數–台灣,62.47,台灣消費者信心指數,2.42,消費者物價指數(CPI)年增率,5482976.0,-3.31,經濟成長率(GDP)–單季
14411,202302,1102,1,102.15,名目有效匯率指數–台灣,62.47,台灣消費者信心指數,2.42,消費者物價指數(CPI)年增率,5482976.0,-3.31,經濟成長率(GDP)–單季
14412,202302,1103,1,102.15,名目有效匯率指數–台灣,62.47,台灣消費者信心指數,2.42,消費者物價指數(CPI)年增率,5482976.0,-3.31,經濟成長率(GDP)–單季


In [ ]:
# Extract test and train data from overall data
#X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
#y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]